lc-biweekly prefix sum & suffix sum, greedy, mask of unique bits


In [ ]:
from typing import List

In [ ]:
# TLE, brute force
# class Solution:
#     def maximumOr(self, nums: List[int], k: int) -> int:
  
#         nums.sort(reverse = True)
#         max_bits = int(math.log(nums[0],2))

#         candidates = [nums[0]]
        
#         for num in nums[1:]:
#             if int(math.log(num,2)) == max_bits:
#                 candidates.append(num)
#             else:
#                 break
                
#         cand = len(candidates)
        
#         rlt = 0
#         for i in range(cand,len(nums)):
#             rlt = rlt | nums[i]

#         max = 0
#         for i in range(cand):
#             temp = rlt
            
#             for num in candidates[0:i]+candidates[i+1:]:
#                 temp = temp | num
                
#             temp = temp | candidates[i]*pow(2,k)
                
#             if temp > max:
#                 max = temp
        
#         return max
        
class Solution:
    def maximumOr(self, nums: List[int], k: int) -> int:
      
        max_bits = max((len(bin(num))-2) for num in nums) 
      
        candidates = [num for num in nums if num >= 2**(max_bits-1)] 
      
        nums = [num for num in nums if num not in candidates]
              
        cand = len(candidates)
      
        rlt = 0
        for num in nums:
            rlt = rlt | num

        max_rlt = 0
        for i in range(cand):
            temp = rlt
          
            for num in candidates[0:i]+candidates[i+1:]:
                temp = temp | num
              
            temp = temp | candidates[i]*pow(2,k)
              
            if temp > max_rlt:
                max_rlt = temp
      
        return max_rlt
  

In [ ]:
#  prefix & suffix sum
# TC 33.33%' 1013 ms
# SC 33.3% 31MB
class Solution:
    def maximumOr(self, nums: List[int], k: int) -> int:
        prefix = [nums[0]]
        for i in range(1,len(nums)):
            prefix.append(prefix[i-1] | nums[i])
            

        suffix = [0 for _ in range(len(nums))]
        suffix[-1] = nums[-1]
        for i in range(len(nums)-2, -1, -1):
            suffix[i] = (suffix[i+1] | nums[i])
        
        max_rlt = 0
        for i in range(len(nums)):
            rlt = (nums[i]*pow(2,k) ) 
            if i>0:
                rlt |= prefix[i-1] 
            if i<len(nums)-1:
                rlt |= suffix[i+1]

            max_rlt = max(max_rlt,rlt)
        
        return max_rlt

In [ ]:
# greedy
# TC 33.33%' 6013 ms
# SC 33.3% 31MB

from collections import Counter
class Solution:
    def maximumOr(self, nums: List[int], k: int) -> int:
        
        # counter[i]: total number of 1s on i-th bit in all elements of nums
        counter = [0]*32  #1 <= nums[i] <= 10^9 < 2^32 
        for num in nums:
            for i in range(32):
                counter[i] += num & 1
                num = num >> 1
        
        rlt = 0
        for j,num in enumerate(nums):
            temp = num
            # calculate the bitwise-or of all without num
            tempcount =  counter.copy()
            for i in range(32):
                tempcount[i] -= temp & 1
                temp = temp >> 1
            # convert bit-counts to int value
            temprlt = 0
            for i in range(32):
                temprlt += (tempcount[i]>0) * (1<<i)
            temprlt = temprlt | (num<<k)
            rlt = max(rlt,temprlt)
        return rlt

In [ ]:
# mask of unique bits
# simliar to greedy, but use uniqueMask to reduce calculation (iteration over counter)
# TC: 2357ms
# SC: 31mb
class Solution:
    def maximumOr(self, nums: List[int], k: int) -> int:
        
        # same as above:
        # counter[i]: total number of 1s on i-th bit in all elements of nums
        counter = [0]*32
        # calculate the bitwise or of all element, used later with uniqueMask
        orSum = 0
        for num in nums:
            orSum |= num
            bit = 0 # this time, not assume list size = 32
            while num:
              if num % 2 == 1:
                counter[bit] += 1 
              num = num >> 1
              bit += 1

        # get the OR of all bits that are unique (1 only in one num)
        # uniqueMask: value of number consisted of all unique bits
        uniqueMask = 0
        for i, count in enumerate(counter):
           if count == 1:
              uniqueMask |= 1 << i
        
        # The best value is produced by left-shifting one num k times. 
        # The left-shifting will remove any bits that num had uniquely
        # added, so remove these first.
        maxrlt = 0
        for num in nums:
          orSumShift = orSum - (num & uniqueMask)
          orSumShift |= num << k
          maxrlt = max(maxrlt, orSumShift)

        return maxrlt 
            
                